In [38]:
import json;
import pandas as pd;
import scipy;
from scipy import stats;
# from apyori import apriori;
import matplotlib.pyplot as plt;
import numpy as np;
import sklearn
from sklearn.datasets import load_iris
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import tree
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
#sia.polarity_scores("")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\13369\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [39]:
data_path = 'okcupid_11_25_22.csv'
df = pd.read_csv(data_path)

In [8]:
#sia.polarity_scores("about me:  i would love to think that i was some some kind of intellectual: either the dumbest smart guy, or the smartest dumb guy. can't say i can tell the difference. i love to talk about ideas and concepts. i forge odd metaphors instead of reciting cliches. like the simularities between a friend of mine's house and an underwater salt mine. my favorite word is salt by the way (weird choice i know). to me most things in life are better as metaphors. i seek to make myself a little better everyday, in some productively lazy way. got tired of tying my shoes. considered hiring a five year old, but would probably have to tie both of our shoes... decided to only wear leather shoes dress shoes.  ")

{'neg': 0.072, 'neu': 0.769, 'pos': 0.159, 'compound': 0.8642}

In [40]:
 # avoid np.nan related errors, default no responses to 0s in polarity_scores() 
df.fillna('', inplace=True)

# create array of the distinct essay columns 
essay_columns = [] 
for i in range(10):
    essay_columns.append(f"essay{i}")

# iterate through essay columns transform them to 4 columns of polarity_scores() values, save new column names
polarity_score_keys = ['neg', 'neu', 'pos', 'compound']
for essay_column in essay_columns:
    sentiment_dict_vec = list(map(lambda x: sia.polarity_scores(x), df[essay_column]))
    for key in polarity_score_keys: #for each key extract its val to make distinct columns
        df[f"{essay_column}_{key}_score"] = np.array(list(map(lambda x: x[key], sentiment_dict_vec)))       

In [14]:
#list(map(lambda x: sia.polarity_scores(x), df["essay0"]))

[{'neg': 0.054, 'neu': 0.718, 'pos': 0.227, 'compound': 0.9933},
 {'neg': 0.076, 'neu': 0.67, 'pos': 0.254, 'compound': 0.9794},
 {'neg': 0.065, 'neu': 0.681, 'pos': 0.254, 'compound': 0.9956},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.07, 'neu': 0.624, 'pos': 0.306, 'compound': 0.9136},
 {'neg': 0.018, 'neu': 0.664, 'pos': 0.318, 'compound': 0.9964},
 {'neg': 0.058, 'neu': 0.495, 'pos': 0.446, 'compound': 0.9679},
 {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 0.787, 'pos': 0.213, 'compound': 0.9492},
 {'neg': 0.025, 'neu': 0.645, 'pos': 0.33, 'compound': 0.995},
 {'neg': 0.043, 'neu': 0.642, 'pos': 0.315, 'compound': 0.9984},
 {'neg': 0.545, 'neu': 0.455, 'pos': 0.0, 'compound': -0.5574},
 {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.052, 'neu'

In [18]:
# Drop columns based on column index to only have age
df_essays = df.drop(df.columns[1:31], axis = 1)
df_essays  

,age,essay0_neg_score,essay0_neu_score,essay0_pos_score,essay0_compound_score,essay1_neg_score,essay1_neu_score,essay1_pos_score,essay1_compound_score,essay2_neg_score,...,essay7_pos_score,essay7_compound_score,essay8_neg_score,essay8_neu_score,essay8_pos_score,essay8_compound_score,essay9_neg_score,essay9_neu_score,essay9_pos_score,essay9_compound_score
0,22,0.054,0.718,0.227,0.9933,0.052,0.722,0.227,0.8176,0.000,...,0.000,0.0000,0.000,1.000,0.000,0.0000,0.093,0.772,0.135,-0.3164
1,35,0.076,0.670,0.254,0.9794,0.190,0.595,0.214,0.0516,0.240,...,0.000,0.0000,0.000,0.784,0.216,0.2960,0.000,0.000,0.000,0.0000
2,38,0.065,0.681,0.254,0.9956,0.075,0.755,0.170,0.8968,0.083,...,0.000,0.0000,0.000,1.000,0.000,0.0000,0.074,0.527,0.398,0.9393
3,23,0.000,1.000,0.000,0.0000,0.417,0.583,0.000,-0.6486,0.000,...,0.000,0.0000,0.000,0.000,0.000,0.0000,0.000,1.000,0.000,0.0000
4,29,0.070,0.624,0.306,0.9136,0.000,0.625,0.375,0.3400,0.000,...,0.000,0.0000,0.000,0.000,0.000,0.0000,0.000,0.000,0.000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59937,59,0.024,0.570,0.406,0.9722,0.037,0.763,0.199,0.9200,0.000,...,0.240,0.6249,0.000,0.909,0.091,0.2500,0.000,0.530,0.470,0.7351
59938,24,0.044,0.829,0.128,0.8909,0.000,1.000,0.000,0.0000,0.000,...,0.000,0.0000,0.000,1.000,0.000,0.0000,0.105,0.726,0.169,0.2023
59939,42,0.014,0.815,0.171,0.9832,0.000,0.645,0.355,0.6705,0.000,...,0.623,0.7650,0.000,0.723,0.277,0.3182,0.000,0.600,0.400,0.2500
59940,27,0.050,0.890,0.059,0.1027,0.000,0.900,0.100,0.4404,0.000,...,0.000,0.0000,0.123,0.521,0.357,0.8070,0.000,0.597,0.403,0.4019


In [24]:
#for c in df_essays.columns[1:]:
    #print(np.corrcoef(df[c], df['age'])[0, 1])

-0.0802453949253237
-0.03942637947927054
0.11733582632642474
0.1210113737457455
-0.028187926281378294
-0.08254308440675061
0.11542346964862375
0.11719175623930757
-0.07051068555799447
-0.012952759256178923
0.06856393674663236
0.09327022688453983
-0.053970759978537855
-0.008719603967412958
0.09898360264528562
0.09384911962254779
-0.12688699930634006
0.01224340435037772
0.0452062175403278
0.09097954650958073
-0.042475720885131046
-0.03386477742135268
0.11510011010499893
0.13089238830451036
-0.027712507470059992
-0.009041391524425709
0.06541554401455993
0.07377136301063698
-0.034684130289883784
0.017624421482141713
-0.01736989599144215
0.013169589228453897
-0.034736866198020855
0.04797262247081125
0.01330251951493249
0.03633466804227163
-0.049340387344205185
-0.0018617029234282735
0.015083938201257696
0.05343281235013393


In [43]:
X = df_essays.filter(items = df_essays.columns[1:], axis=1).values
y = df_essays['age'].values
y=y.astype('int')
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=14)

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
standard_scaler = StandardScaler()

pca_transformer10 = PCA(n_components=10)
X_PCA_10 = standard_scaler.fit_transform(X)
X_PCA_10 = pca_transformer10.fit_transform(X_PCA_10)
lin_reg_pca10 = LogisticRegression()
lin_reg_pca10.fit(X_PCA_10, y) 

X_train_PCA_10, X_test_PCA_10, y_train_PCA_10, y_test_PCA_10 = train_test_split(X_PCA_10,y,test_size=0.2, random_state=14)

pca_transformer5 = PCA(n_components=5)
X_PCA_5 = standard_scaler.fit_transform(X)
X_PCA_5 = pca_transformer5.fit_transform(X_PCA_5)
lin_reg_pca5 = LogisticRegression()
lin_reg_pca5.fit(X_PCA_5, y) 

X_train_PCA_5, X_test_PCA_5, y_train_PCA_5, y_test_PCA_5 = train_test_split(X_PCA_5,y,test_size=0.2, random_state=14)

pca_transformer15 = PCA(n_components=15)
X_PCA_15 = standard_scaler.fit_transform(X)
X_PCA_15 = pca_transformer15.fit_transform(X_PCA_15)
lin_reg_pca15 = LogisticRegression()
lin_reg_pca15.fit(X_PCA_15, y) 

X_train_PCA_15, X_test_PCA_15, y_train_PCA_15, y_test_PCA_15 = train_test_split(X_PCA_15,y,test_size=0.2, random_state=14)

C:\Users\13369\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\13369\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

In [47]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
regr5 = linear_model.LinearRegression()
regr10 = linear_model.LinearRegression()
regr15 = linear_model.LinearRegression()

regr.fit(X_train, y_train)
regr5.fit(X_train_PCA_5, y_train_PCA_5)
regr10.fit(X_train_PCA_10, y_train_PCA_10)
regr15.fit(X_train_PCA_15, y_train_PCA_15)

LinearRegression()

In [48]:
y_preds_lin_reg = regr.predict(X_test)
y_preds_lin_reg5 = regr5.predict(X_test_PCA_5)
y_preds_lin_reg10 = regr10.predict(X_test_PCA_10)
y_preds_lin_reg15 = regr15.predict(X_test_PCA_15)

from sklearn import metrics

print(metrics.r2_score(y_test, y_preds_lin_reg))
print(metrics.r2_score(y_test, y_preds_lin_reg5))
print(metrics.r2_score(y_test, y_preds_lin_reg10))
print(metrics.r2_score(y_test, y_preds_lin_reg15))

0.08106230855577234
0.04966956900678465
0.06242640417017287
0.0659184459129829
